### Heroes Of Pymoli Data Analysis
* Of the 1163 active players, the vast majority are male (84%). There also exists, a smaller, but notable proportion of female players (14%).

* Our peak age demographic falls between 20-24 (44.8%) with secondary groups falling between 15-19 (18.60%) and 25-29 (13.4%).  
-----

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [6]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File Name to Open
file_of_interest = "purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_of_interest)
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


## Player Count

* Display the total number of players


In [7]:
#drop duplicate SNs or people with multiple purchases
uniqueplayerlist = purchase_data["SN"].drop_duplicates()
uniqueplayercount = uniqueplayerlist.count()
uniqueplayercount




576

## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [8]:
#drop duplicate 'Item IDs' that have multiple purchases associated with them, and then count unique items
uniqueitemlist = purchase_data["Item ID"].drop_duplicates()
uniqueitemcount = uniqueitemlist.count()
# Calculate the total amount associated with item purchases 
totalrevenue = purchase_data["Price"].sum()
#Calculate the total number of item purchases
totalnumberofpurchases = (purchase_data["Price"]).count()
#Calculate the average price associated with a unique item
avgprice = round(totalrevenue/ totalnumberofpurchases,2)
# Place all of the data found into a summary DataFrame
purchasing_analysis = pd.DataFrame({"Total unique items purchased": uniqueitemcount,
                              "Average Purchase Price ": avgprice,
                              "Total Purchases": [totalnumberofpurchases],
                              "Total Revenue": [totalrevenue]})
purchasing_analysis





,Total unique items purchased,Average Purchase Price,Total Purchases,Total Revenue
0,183,3.05,780,2379.77


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [9]:
#calculate gender totals without the duplicates using uniqueplayerlist from above
# "purchase_data.drop_duplicates(subset="SN")" only keeps unique values in "SN" column and keeps rest of the purchase_data available 
uniquemalecounts = purchase_data.drop_duplicates(subset="SN")["Gender"].value_counts()["Male"]
uniquefemalecounts = purchase_data.drop_duplicates(subset="SN")["Gender"].value_counts()["Female"]
uniqueothercounts = purchase_data.drop_duplicates(subset="SN")["Gender"].value_counts()["Other / Non-Disclosed"]


#calculate gender percentages and round result to 2 decimal places
malepercent = uniquemalecounts/[uniqueplayerlist.count()] * 100
femalepercent = uniquefemalecounts/[uniqueplayerlist.count()] * 100
otherpercent = uniqueothercounts/[uniqueplayerlist.count()] * 100

#Then round the number to 2 decimal places
malepercent = malepercent.round(2)
femalepercent = femalepercent.round(2)
otherpercent = otherpercent.round(2)

# Place all of the data found into a summary DataFrame
gender_demographics = pd.DataFrame({"Gender(s)": ["Male", "Female", "Other / Non-Disclosed"],
                              "Player Count": [uniquemalecounts,uniquefemalecounts,uniqueothercounts],
                              "Percentage": [malepercent,femalepercent,otherpercent]})

#Change percent number to string and add % sign
gender_demographics["Percentage"] = gender_demographics["Percentage"].astype(str) + '%'

#print summary table
gender_demographics


    

,Gender(s),Player Count,Percentage
0,Male,484,[84.03]%
1,Female,81,[14.06]%
2,Other / Non-Disclosed,11,[1.91]%



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [10]:
#Create the groupby object to segregate by gender
gender_group = purchase_data.groupby(["Gender"])

#calculate per gender values for purchase count, avg. purchase price, total purchase, avg. purchase price
purchase_count = gender_group["Purchase ID"].count()
purchase_total = gender_group["Price"].sum()
avg_purch_price = purchase_total/purchase_count
#round the price to 2 decimals
roundedavgpurch = round(avg_purch_price,2)

#Place initial data found above into a summary DataFrame
purch_analysis = pd.DataFrame({"Gender(s)": ["Male", "Female", "Other / Non-Disclosed"],
                                "Purchase Count": purchase_count, 
                                   "Purchase Total": purchase_total,
                                    "Average Purchase Price": roundedavgpurch})
#purch_analysis

#merge purch_analysis and gender_demographcs to bring the uniqe male/female/other count in the same dataframe for avg purchase price per person
purch_gender = pd.merge(purch_analysis,gender_demographics,on=["Gender(s)"])
#calculate avg. purchase price per person
avg_pppp = purch_gender["Purchase Total"]/purch_gender["Player Count"]
#round pirce to 2 decimal places
avg_pppp_rd = round(avg_pppp,2)
#add avg purchase price per person as a new column to the purch_gender dataframe
purch_gender["Avg. Purchase Price Per Person"] = avg_pppp_rd
#now delete the player count and percentage columns to generate the final gender-specific purchasing analysis
purch_gender_final = purch_gender[['Gender(s)', 'Purchase Count', 'Purchase Total',
       'Average Purchase Price', 'Avg. Purchase Price Per Person']]

purch_gender_final



,Gender(s),Purchase Count,Purchase Total,Average Purchase Price,Avg. Purchase Price Per Person
0,Male,113,361.94,3.20,0.75
1,Female,652,1967.64,3.02,24.29
2,Other / Non-Disclosed,15,50.19,3.35,4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [11]:

#drop rows containing duplicate SNs to get unique count
raw_data = purchase_data.drop_duplicates(subset="SN")


#create a bin, group labels, and use pd.cut to assign age groups
bins = [0, 10, 15, 20, 25, 30, 35, 40, 100]
group_labels = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-40","40+"]
age_demographics = pd.cut(raw_data["Age"], bins, labels=group_labels)

 
#make the result look pretty
df = pd.DataFrame(age_demographics.value_counts())

df.index.name="Age Group"
df.columns=["Count"]

df

,Count
Age Group,
20-24,232
15-19,150
25-29,59
10-14,41
30-34,37
35-40,26
<10,24
40+,7


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [231]:
#use the age_demographics binning and add a column to the original purchase_data
purchase_data["Age Group"] = age_demographics
#create a new dataframe by using the groupby and 'Age Group Bin' column 
purch_age = purchase_data.groupby("Age Group")
#calculate purchase count, total, and avg. price by age group bins
purch_count = purch_age["Purchase ID"].count()
purch_total = round(purch_age["Price"].sum(),2)
avgpurchpriceage= round(purch_total/purch_count,2)
purch_age_analysis=pd.DataFrame({"Purchase Count":purch_count, "Purchase Total": purch_total, "Average Purchase Price":avgpurchpriceage})
purch_age_analysis



,Purchase Count,Purchase Total,Average Purchase Price
Age Group,,,
<10,24,82.18,3.42
10-14,41,120.43,2.94
15-19,150,476.84,3.18
20-24,232,700.03,3.02
25-29,59,171.02,2.90
30-34,37,111.15,3.00
35-40,26,85.47,3.29
40+,7,21.53,3.08


In [12]:
purch_topspenders = purchase_data.groupby("SN")
purch_spendertotal=purch_topspenders["Price"].sum()
purch_spendercount=purch_topspenders["Price"].count()
purch_spenderavg=round(purch_spendertotal/purch_spendercount,2)
df2=pd.DataFrame({"Purchase Total":purch_spendertotal,"Purchase_Count":purch_spendercount,"Average Purchase Price":purch_spenderavg})
df2.head()


,Purchase Total,Purchase_Count,Average Purchase Price
SN,,,
Adairialis76,2.28,1,2.28
Adastirin33,4.48,1,4.48
Aeda94,4.91,1,4.91
Aela59,4.32,1,4.32
Aelaria33,1.79,1,1.79


* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [13]:
df2.sort_values(by=['Purchase Total'],inplace=True, ascending=False)
df2.head()

,Purchase Total,Purchase_Count,Average Purchase Price
SN,,,
Lisosia93,18.96,5,3.79
Idastidru52,15.45,4,3.86
Chamjask73,13.83,3,4.61
Iral74,13.62,4,3.40
Iskadarya95,13.10,3,4.37


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [14]:
popular_items = purchase_data.groupby(["Item ID","Item Name"])
purch_count_items = popular_items["Item Name"].count()
purch_value_items = popular_items["Price"].sum()
purch_item_prices = round(purch_value_items/purch_count_items,2)

items = pd.DataFrame({"Purchase Count":purch_count_items,"Item Price":purch_item_prices,"Total Purchase Value":purch_value_items})
items.head()
                                      


,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
0,Splinter,4,1.28,5.12
1,Crucifer,3,3.26,9.78
2,Verdict,6,2.48,14.88
3,Phantomlight,6,2.49,14.94
4,Bloodlord's Fetish,5,1.70,8.50


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [15]:
items.sort_values(by=['Total Purchase Value'],inplace=True, ascending=False)
items.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,4.23,50.76
82,Nirvana,9,4.90,44.10
145,Fiery Glass Crusader,9,4.58,41.22
92,Final Critic,8,4.88,39.04
103,Singed Scalpel,8,4.35,34.80
